In [ ]:
import cv2

stream = cv2.VideoCapture(0)

In [ ]:
import tensorflow as tf

panis = tf.keras.models.load_model("mymodelv1.4.h5")

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
import numpy as np

def preproc(img):
    img = img / 255.0
    img = tf.image.resize(img, [128, 128])
    return img

In [ ]:
def score_frame(frame, model):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    frame = [frame]
    results = model(frame)
    labels = results.xyxyn[0][:, -1].numpy()
    cord = results.xyxyn[0][:, :-1].numpy()
    return labels, cord

In [ ]:
def plot_boxes(results, frame, model):
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]
    for i in range(n):
        row = cord[i]
        # If score is less than 0.2 we avoid making a prediction.
        if row[4] < 0.2: 
            continue
        x1 = int(row[0]*x_shape)
        y1 = int(row[1]*y_shape)
        x2 = int(row[2]*x_shape)
        y2 = int(row[3]*y_shape)
        
        img = frame[y1:y2, x1:x2]
        img = preproc(img)
        img = tf.expand_dims(img, axis=0)
        predictions = panis.predict(img)
        predictions = predictions.flatten().tolist()
        
        pred_label = 'rotten' if predictions[0] > 0.5 else 'fresh'
        bgr = (0, 0, 255) if predictions[0] > 0.5 else (0, 255, 0)
        
        classes = model.names # Get the name of label index
        label_font = cv2.FONT_HERSHEY_SIMPLEX #Font for the label.
        cv2.rectangle(frame, \
                      (x1, y1), (x2, y2), \
                       bgr, 2) #Plot the boxes
        cv2.putText(frame,\
                    pred_label, \
                    (x1, y1), \
                    label_font, 0.9, bgr, 2) #Put a label over box.
        return frame

In [ ]:
x_shape = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
y_shape = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
tfc = int(stream.get(cv2.CAP_PROP_FRAME_COUNT))
four_cc = cv2.VideoWriter_fourcc(*"MJPG")
out = cv2.VideoWriter('sample.avi', four_cc, 20, (x_shape, y_shape))

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, Image
import ipywidgets as widgets
import threading

def view():
    display_handle=display(None, display_id=True)
    while True:
        ret, img = stream.read()
        if ret is None:
            continue
        results = score_frame(img, model)
        res = plot_boxes(results, img, model)
        try:
            _, frame = cv2.imencode('.jpeg', res)
        except:
            pass
        display_handle.update(Image(data=frame.tobytes()))

thread = threading.Thread(target=view)
thread.start()

In [ ]:
stream.release()

In [ ]:
torch.cuda.is_available()